In [1]:
from __future__ import division
import tensor_calculation as t
import tensor_advanced as ta
import random
from scipy import stats 
from scipy.stats import multivariate_normal as m_n
import numpy as np
import data_whitening as dw
from  math import log

In [2]:
def vec_mean(v):
    return t.list_nu( t.list_sum(v), 1/max(1, len(v)))

In [3]:
def dist(v,u):
    return t.list_contrac(t.list_sub(v, u), t.list_sub(v, u))**0.5

In [4]:
def make_mat(u, v):
    return ta.mat_dot(ta.colu_fi(u), [v])

In [5]:
def trace(m):
    dig = [m[i][i] for i in range(len(m))]
    return sum(dig)

In [6]:
def ge_pi (k):           #generate a multinomial
    l = [random.randint(0, 10000) for i in range(k-1)]
    l_p = sorted(l)+[10000]
    l_m =[0]+sorted(l)
    l_re = [i-j for (i,j) in zip(l_p , l_m)]
    return [i/10000 for i in l_re]

In [7]:
def ge_pi2 (k):
    l = [random.randint(0, 10000/(k-1)) for i in range(k-1)]
    l_re=[ i-j+(10000/(k-1))  for (i,j) in zip(l+[0], [10000/(k-1)]+l)]
    return [ i/10000 for i in l_re]

In [8]:
def ge_cov(k):    #generate a covariance matrix
    data_x=ta.gene_random_tensor([10,k], (-1, 1))
    return dw.covari(data_x)

In [22]:
def comp_gam(data, c, cov, pi):           #compute gama from data
    gam_data=[[m_n.pdf(i, j, k)*l for (j,k,l) in zip(c, cov, pi)] for i in data]
    return t.transp([[j/sum(i) for j in i] for i in gam_data], [1,0] )#i is a list 

In [10]:
def c_upd(data, gam, n_k):           #updata c
    data1 = [[t.list_nu(j,k) for (j,k) in zip(data, i) ]  for i in gam]
    return [t.list_nu(t.list_sum(i), 1/j) for (i,j) in zip(data1, n_k)]

In [11]:
def cov_upd(data, c, gma, n_k):     #update all the covariance at once
    data1 = [[t.list_sub(j, i)for j in data] for i in c]
    data2=[[t.list_nu(l,k) for (k,l) in zip(i,j)] for (i,j) in zip(gma, data1)]  #i,j are lists
    mat_unsum=[[make_mat(i, j) for (i,j) in zip(k,l)] for (k,l) in zip(data1, data2)]
    mat =[t.list_sum(i) for i in mat_unsum]
    return [t.list_nu( j, 1/i) for (i,j) in zip(n_k, mat)]

In [12]:
def lik_hod(data, c, cov_l, pi):
    data_likhod = [ sum([m_n.pdf(i, j, k)*l for (j,k,l) in zip (c, cov_l, pi) ]) for i in data]
    return -1*log(t.list_mu(data_likhod))

In [23]:
def m_g (data , k):
    dem = len(data[0])
    para_list=[]
    for i in range(len(data)//100+5):
        c =random.sample(data, k)
        pi = ge_pi(k)
        cov_l = [ge_cov(dem) for i in range(k)]
        while True:
            gam = comp_gam(data, c, cov_l, pi)
            n_k = [sum(i) for i in gam]
            cnew = c_upd(data, gam, n_k)
            cov_l = cov_upd(data, c, gam, n_k)
            pi = [i/sum(n_k) for i in n_k]
            if max([dist(i,j)/(max(ta.mat_dot(i,i)**0.5, 0.00001))  for (i,j) in zip(c, cnew) ]) < 000000.1:
                break
            c = cnew
        para_list.append([c, cov_l, pi])
    lh = [lik_hod(data, i[0], i[1], i[2]) for i in para_list]
    ind = lh.index(max(lh))
    return para_list[ind]

In [14]:
data_k=ta.gene_random_tensor([1000,2], (-2, 2))
data =[]
for dk in data_k:
    if (dk[0]-1)**2+ dk[1]**2<0.5**2 or (dk[0]+1)**2+ dk[1]**2<0.5**2:
        data.append(dk)

In [25]:
m_g(data, 2)

[[[0.82080458908991394, 0.010283837468315786],
  [-1.0111601782480102, -0.017749597949407805]],
 [[[0.068253779943397569, -0.0086736385208092681],
   [-0.0086736385208092681, 0.063905896483831701]],
  [[0.066793003573036483, 0.00028856499824927133],
   [0.00028856499824926976, 0.074762540470991498]]],
 [0.45903693675346641, 0.54096306324653354]]